<a href="https://colab.research.google.com/github/Avnish3909/Resume-Classifier-NLP/blob/main/Resume_Classifier_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install spacy pdfminer.six


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.9 MB/s eta 0:00:00


In [2]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,501 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [3]:
!pip install pytesseract

In [4]:
!apt-get install poppler-utils
!apt-get install libpoppler-cpp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 1s (186 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120921 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW p

In [5]:
!pip install pdf2image

In [27]:
import os
import pytesseract
import docx2txt
from pdf2image import convert_from_path
from PIL import Image
import re
import spacy
from google.colab import files

nlp = spacy.load("en_core_web_sm")

predefined_skills = [
    "python", "java", "javascript", "sql", "html", "css", "react",
    "machine learning", "data analysis", "data science", "deep learning",
    "database management", "web development", "cloud computing",
    "software development", "devops", "docker", "kubernetes", "agile",
    "android", "ios", "artificial intelligence", "big data", "c++",
    "c#", "php", "ruby", "angular", "typescript", "node.js", "git",
    "frontend", "backend", "full-stack", "ux/ui design", "blockchain",
    "cybersecurity", "networking", "shell scripting", "wordpress"
]

EDUCATION = [
    'BE', 'B.E.', 'B.E', 'BS', 'B.S',
    'ME', 'M.E', 'M.E.', 'MS', 'M.S',
    'BTECH', 'B.TECH', 'M.TECH', 'MTECH',
    'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
]

email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0.9.-]+\.[A-Z|a-z]{2,7}\b'
phone_pattern = r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b'

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text

def extract_text_from_docx(docx_path):
    extracted_text = docx2txt.process(docx_path)
    return extracted_text

def extract_text_from_pdf(pdf_path):
    pdf_images = convert_from_path(pdf_path)
    extracted_text = ""
    for pdf_image in pdf_images:
        text = pytesseract.image_to_string(pdf_image)
        extracted_text += text
    return extracted_text

def getSkills(resume_text, st_en):
    nlp_text = st_en(resume_text)
    noun_chunks = nlp_text.noun_chunks
    tokens = [token.text for token in nlp_text if not token.is_stop]

    skillset = []

    for token in tokens:
        if token.lower() in predefined_skills:
            skillset.append(token)

    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in predefined_skills:
            skillset.append(token)

    return [i.capitalize() for i in set([i.lower() for i in skillset])]

def getEducation(resume_text, st_en):
    nlp_text = st_en(resume_text)
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}

    for index, text in enumerate(nlp_text):
        for tex in text.split():
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper in EDUCATION and not st_en.vocab[tex].is_stop:
                edu[tex] = text + nlp_text[index]

    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)

    return education

def getExperience(resume_text, st_en):
    nlp_text = st_en(resume_text)
    tokens = [token.text.lower() for token in nlp_text if not token.is_stop]

    experience_keywords = ['experience', 'work', 'job', 'career']

    for keyword in experience_keywords:
        if keyword in tokens:
            return "Experienced"

    return "Not Experienced"

def getName(resume_text, st_en):
    nlp_text = st_en(resume_text)
    matcher = Matcher(st_en.vocab)

    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME', [pattern])

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

def getContactDetails(resume_text):
    emails = re.findall(email_pattern, resume_text)
    phone_numbers = re.findall(phone_pattern, resume_text)

    contact_details = {
        "Emails": emails,
        "Phone Numbers": phone_numbers
    }

    return contact_details

uploaded = files.upload()

if len(uploaded) > 0:
    file_name = list(uploaded.keys())[0]

    with open(file_name, 'wb') as file:
        file.write(uploaded[file_name])

    if file_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
        extracted_text = extract_text_from_image(file_name)
    elif file_name.lower().endswith((".doc", ".docx")):
        extracted_text = extract_text_from_docx(file_name)
    elif file_name.lower().endswith((".pdf")):
        extracted_text = extract_text_from_pdf(file_name)

    education_info = getEducation(extracted_text, nlp)
    skills_info = getSkills(extracted_text, nlp)
    contact_details = getContactDetails(extracted_text)
    name_info = getName(extracted_text, nlp)
    experience_info = getExperience(extracted_text, nlp)

    resume_info = {
        'Name': name_info,
        'Education': education_info,
        'Skills': skills_info,
        'Experience': experience_info,
        'Contact Details': contact_details
    }

    print(f"Resume Info:")
    print(f"Name: {resume_info['Name']}")
    print(f"Education: {resume_info['Education']}")
    print(f"Skills: {resume_info['Skills']}")
    print(f"Experience: {resume_info['Experience']}")
    print(f"Contact Details: {resume_info['Contact Details']}")

    os.remove(file_name)
else:
    print("No files were uploaded.")


Saving Jagan S Iyer PM.docx to Jagan S Iyer PM.docx
Resume Info:
Name: Jagan S
Education: []
Skills: ['Cloud computing', 'Agile', 'Css', 'Sql']
Experience: Experienced
Contact Details: {'Emails': ['jagansrconsult@gmail.com', 'jagansrconsult@gmail.com'], 'Phone Numbers': []}


In [19]:
!pip install docx2txt